# TOI Scraper
### Zach Andrews
### ztandrews18@sbcglobal.net
### github.com/ztandrews
---
##### Scrape NHL Player Even-Strength TOI for a specific game

In [45]:
#Import modules
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
#Config
game_id = "" # Game ID, excluding 0. Find more on this from the NHL api at https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md#game-ids. ex. 20879
home_abv = "" # Home team's abbreviation
away_abv = "PIT" # Away team's abbreviation
game_date = "0217" # Month, Day. ex. 0217 for feb 17. Not strict

In [47]:
#Home team block
home_URL = "https://www.nhl.com/scores/htmlreports/20222023/TH0"+game_id+".HTM"
home_page = requests.get(home_URL)
home_soup = bs(home_page.content, "html.parser")
home_player_search = home_soup.find_all("td",class_="playerHeading + border")
home_players = []
for p in home_player_search:
    home_players.append(p.string)
home_names = []
for player in home_players:
    p_name = player.split(' ')
    name = (p_name[2],p_name[1])
    name = str(name)
    final_name = name.replace(',','')
    final_name = final_name.replace('(','')
    final_name = final_name.replace(')','')
    final_name = final_name.replace("'",'')
    home_names.append(final_name)
home_toi_search = home_soup.find_all("td",class_="bborder + lborder + bold")
home_tois = []
for i in range(1,len(home_toi_search)):
    if i==len(home_toi_search):
        continue
    if(str(home_toi_search[i].string)=='TOT'):
        home_tois.append(home_toi_search[i-2].string)
home_tois.append(home_toi_search[len(home_toi_search)-2].string)
home_dec_tois = []
HMS = [60*60, 60, 1]
for t in home_tois:
    dec_time = sum(a * b for a,b in zip(HMS, map(int, t.split(":"))))
    dec_time /= 3600.
    home_dec_tois.append(dec_time)
home_toi_df = pd.DataFrame(columns=['Player','TOI'])
for k in range(0,len(home_player_search)):
    home_toi_df = home_toi_df.append({'Player':home_names[k],'TOI':home_dec_tois[k]},ignore_index=True)
home_toi_df

,Player,TOI
0,ADAM PELECH,21.466667
1,RYAN PULOCK,20.416667
2,NOAH DOBSON,16.116667
3,SIMON HOLMSTROM,6.366667
4,ZACH PARISE,17.050000
5,JOSH BAILEY,6.783333
6,MATHEW BARZAL,19.550000
7,BO HORVAT,21.966667
8,MATT MARTIN,11.750000
9,ANDY ANDREOFF,5.083333


In [48]:
#Save home team to CSV
home_toi_df.to_csv(home_abv+"_ev_toi_"+game_date+".csv")

In [49]:
#Away team block
away_URL = "https://www.nhl.com/scores/htmlreports/20222023/TV0"+game_id+".HTM"
away_page = requests.get(away_URL)
away_soup = bs(away_page.content, "html.parser")
away_player_search = away_soup.find_all("td",class_="playerHeading + border")
away_players = []
for p in away_player_search:
    away_players.append(p.string)
away_names = []
for player in away_players:
    p_name = player.split(' ')
    name = (p_name[2],p_name[1])
    name = str(name)
    final_name = name.replace(',','')
    final_name = final_name.replace('(','')
    final_name = final_name.replace(')','')
    final_name = final_name.replace("'",'')
    away_names.append(final_name)
away_toi_search = away_soup.find_all("td",class_="bborder + lborder + bold")
away_tois = []
for i in range(1,len(away_toi_search)):
    if i==len(away_toi_search):
        continue
    if(str(away_toi_search[i].string)=='TOT'):
        away_tois.append(away_toi_search[i-2].string)
away_tois.append(away_toi_search[len(away_toi_search)-2].string)
away_dec_tois = []
HMS = [60*60, 60, 1]
for t in away_tois:
    dec_time = sum(a * b for a,b in zip(HMS, map(int, t.split(":"))))
    dec_time /= 3600.
    away_dec_tois.append(dec_time)
away_toi_df = pd.DataFrame(columns=['Player','TOI'])
for k in range(0,len(away_player_search)):
    away_toi_df = away_toi_df.append({'Player':away_names[k],'TOI':away_dec_tois[k]},ignore_index=True)
away_toi_df

,Player,TOI
0,CASEY DESMITH,54.100000
1,CHAD RUHWEDEL,12.016667
2,BRIAN DUMOULIN,20.450000
3,"DREW ""OCONNOR""",10.583333
4,JOSH ARCHIBALD,10.833333
5,JASON ZUCKER,16.866667
6,BRYAN RUST,16.333333
7,BROCK MCGINN,9.250000
8,JEFF PETRY,20.400000
9,MARCUS PETTERSSON,21.000000


In [50]:
#Save away team to CSV
away_toi_df.to_csv(away_abv+"_ev_toi_"+game_date+".csv")